In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)

In [3]:
pyg_dataset = torch.load('../../data/curated dataset/frag_pyg_dataset.pth')

C:\Users\sarab\AppData\Local\Temp\ipykernel_14700\3101895145.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load('../../data/curated dataset/frag_py

In [4]:
node_features = pyg_dataset[0].x.shape[1]  # Should be 134 based on your description
edge_features = pyg_dataset[0].edge_attr.shape[1]  # Should be 6 based on your description

In [5]:
# Define VGAE model
class VGAE(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(VGAE, self).__init__()
        self.encoder = VGEncoder(node_features, edge_features, hidden_dim, latent_dim)
        self.decoder = VGDecoder(latent_dim, hidden_dim, node_features, edge_features)

    def encode(self, x, edge_index, edge_attr):
        return self.encoder(x, edge_index, edge_attr)

    def decode(self, z, edge_index):
        return self.decoder(z, edge_index)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x, edge_index, edge_attr):
        mu, logvar = self.encode(x, edge_index, edge_attr)
        z = self.reparameterize(mu, logvar)
        reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index)
        return reconstructed_x, reconstructed_edge_attr, mu, logvar

    def generate_multiple_outputs(self, x, edge_index, edge_attr, num_samples=5):
        outputs = []
        with torch.no_grad():
            z, _ = self.encode(x, edge_index, edge_attr)
            for _ in range(num_samples):
                # Add noise to z (sampling from normal distribution)
                z_noisy = z + torch.randn_like(z) * 0.5  # Adjust std deviation for noise level
                reconstructed_x, reconstructed_edge_attr = self.decode(z_noisy, edge_index)
                outputs.append((reconstructed_x, reconstructed_edge_attr, edge_index))
        return outputs

class VGEncoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(VGEncoder, self).__init__()
        self.conv1 = SAGEConv(node_features + edge_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, hidden_dim)  # Additional layer
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, edge_index, edge_attr):
        edge_attr_expanded = torch.zeros((x.size(0), edge_attr.size(1))).to(x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        x = torch.cat([x, edge_attr_expanded], dim=1)

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))  # Added layer

        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)

        return mu, logvar

class VGDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(VGDecoder, self).__init__()
        self.fc = nn.Linear(latent_dim, hidden_dim)
        self.node_decoder = nn.Linear(hidden_dim, node_features)
        self.edge_decoder = nn.Linear(hidden_dim * 2, edge_features)

    def forward(self, z, edge_index):
        h = F.relu(self.fc(z))
        reconstructed_x = self.node_decoder(h)

        # Edge reconstruction
        row, col = edge_index
        edge_h = torch.cat([h[row], h[col]], dim=1)
        reconstructed_edge_attr = self.edge_decoder(edge_h)

        return reconstructed_x, reconstructed_edge_attr

In [6]:
def loss_function(recon_x, x, recon_edge_attr, edge_attr, mu, logvar):
    # Reconstruction losses
    node_recon_loss = F.mse_loss(recon_x, x)
    edge_recon_loss = F.mse_loss(recon_edge_attr, edge_attr)

    # KL divergence
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return node_recon_loss + edge_recon_loss + kl_divergence


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=60

In [8]:
import optuna
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader as PyGDataLoader

# Define the VGAE model (same as before)
class VGAE(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(VGAE, self).__init__()
        self.encoder = VGEncoder(node_features, edge_features, hidden_dim, latent_dim)
        self.decoder = VGDecoder(latent_dim, hidden_dim, node_features, edge_features)

    def encode(self, x, edge_index, edge_attr):
        return self.encoder(x, edge_index, edge_attr)

    def decode(self, z, edge_index):
        return self.decoder(z, edge_index)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x, edge_index, edge_attr):
        mu, logvar = self.encode(x, edge_index, edge_attr)
        z = self.reparameterize(mu, logvar)
        reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index)
        return reconstructed_x, reconstructed_edge_attr, mu, logvar

    def generate_multiple_outputs(self, x, edge_index, edge_attr, num_samples=5):
        outputs = []
        with torch.no_grad():
            z, _ = self.encode(x, edge_index, edge_attr)
            for _ in range(num_samples):
                # Add noise to z (sampling from normal distribution)
                z_noisy = z + torch.randn_like(z) * 0.5  # Adjust std deviation for noise level
                reconstructed_x, reconstructed_edge_attr = self.decode(z_noisy, edge_index)
                outputs.append((reconstructed_x, reconstructed_edge_attr, edge_index))
        return outputs

class VGEncoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(VGEncoder, self).__init__()
        self.conv1 = SAGEConv(node_features + edge_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, hidden_dim)  # Additional layer
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, edge_index, edge_attr):
        edge_attr_expanded = torch.zeros((x.size(0), edge_attr.size(1))).to(x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        x = torch.cat([x, edge_attr_expanded], dim=1)

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))  # Added layer

        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)

        return mu, logvar

class VGDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(VGDecoder, self).__init__()
        self.fc = nn.Linear(latent_dim, hidden_dim)
        self.node_decoder = nn.Linear(hidden_dim, node_features)
        self.edge_decoder = nn.Linear(hidden_dim * 2, edge_features)

    def forward(self, z, edge_index):
        h = F.relu(self.fc(z))
        reconstructed_x = self.node_decoder(h)

        # Edge reconstruction
        row, col = edge_index
        edge_h = torch.cat([h[row], h[col]], dim=1)
        reconstructed_edge_attr = self.edge_decoder(edge_h)

        return reconstructed_x, reconstructed_edge_attr

# Loss function with KL divergence
def loss_function(recon_x, x, recon_edge_attr, edge_attr, mu, logvar):
    node_recon_loss = F.mse_loss(recon_x, x)
    edge_recon_loss = F.mse_loss(recon_edge_attr, edge_attr)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return node_recon_loss + edge_recon_loss + kl_divergence

# Objective function for Optuna to optimize the hyperparameters
def objective(trial):
    # Define hyperparameters to search
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8,16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)

    # Prepare data loaders
    train_ratio = 0.8
    valid_ratio = 0.2

    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train

    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])
    train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = PyGDataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = VGAE(node_features, edge_features, hidden_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience // 2)

    best_val_loss = float('inf')
    epochs_without_improvement = 0

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            optimizer.zero_grad()
            recon_x, recon_edge_attr, mu, logvar = model(data.x, data.edge_index, data.edge_attr)

            loss = loss_function(recon_x, data.x, recon_edge_attr, data.edge_attr, mu, logvar)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        # Validation step
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for val_data in val_loader:
                recon_x_val, recon_edge_attr_val, mu_val, logvar_val = model(val_data.x, val_data.edge_index, val_data.edge_attr)
                total_val_loss += loss_function(recon_x_val, val_data.x, recon_edge_attr_val, val_data.edge_attr, mu_val, logvar_val).item()

        avg_val_loss = total_val_loss / len(val_loader)

        # Print training and validation losses
        print(f"[Trial {trial.number}] Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"[Trial {trial.number}] Early stopping after {epoch + 1} epochs.")
            break

        scheduler.step(avg_val_loss)

    return best_val_loss

# Run the study with Optuna
if __name__ == "__main__":
    import multiprocessing

    num_jobs = min(multiprocessing.cpu_count(), 7)  # Use up to 4 cores
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=num_jobs)

    best_trial = study.best_trial
    print(f"\nBest Trial Number: {best_trial.number}")
    print("Best Parameters:", best_trial.params)
    print("Best Value (Objective):", best_trial.value)


c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-12-06 10:42:51,983] A new study created in memory with name: no-name-c2bb1d01-ee04-44ce-b420-3b0d6b528140


[Trial 0] Epoch 1/60, Training Loss: 3.6297, Validation Loss: 0.2238
[Trial 6] Epoch 1/60, Training Loss: 3895.1328, Validation Loss: 0.2734
[Trial 3] Epoch 1/60, Training Loss: 9.3771, Validation Loss: 0.9101
[Trial 4] Epoch 1/60, Training Loss: 1.2932, Validation Loss: 0.2659
[Trial 0] Epoch 2/60, Training Loss: 0.1337, Validation Loss: 0.1926
[Trial 6] Epoch 2/60, Training Loss: 0.1717, Validation Loss: 0.1866
[Trial 1] Epoch 1/60, Training Loss: 0.6068, Validation Loss: 0.2283
[Trial 3] Epoch 2/60, Training Loss: 0.6096, Validation Loss: 0.5453
[Trial 4] Epoch 2/60, Training Loss: 0.1722, Validation Loss: 0.2328
[Trial 0] Epoch 3/60, Training Loss: 0.1296, Validation Loss: 0.1722
[Trial 5] Epoch 1/60, Training Loss: 0.8370, Validation Loss: 0.2761
[Trial 6] Epoch 3/60, Training Loss: 0.1443, Validation Loss: 0.1810
[Trial 3] Epoch 3/60, Training Loss: 0.3873, Validation Loss: 0.4226
[Trial 4] Epoch 3/60, Training Loss: 0.1531, Validation Loss: 0.2209
[Trial 0] Epoch 4/60, Training 

[I 2024-12-06 10:50:07,825] Trial 0 finished with value: 0.12777193387349448 and parameters: {'hidden_dim': 256, 'latent_dim': 32, 'learning_rate': 0.0032363994918188973, 'batch_size': 64, 'patience': 8}. Best is trial 0 with value: 0.12777193387349448.


[Trial 4] Epoch 27/60, Training Loss: 0.1277, Validation Loss: 0.1396
[Trial 0] Epoch 33/60, Training Loss: 0.1251, Validation Loss: 0.1278
[Trial 0] Early stopping after 33 epochs.
[Trial 1] Epoch 14/60, Training Loss: 0.1273, Validation Loss: 0.1428
[Trial 7] Epoch 1/60, Training Loss: 13.0969, Validation Loss: 0.4191
[Trial 6] Epoch 33/60, Training Loss: 0.1284, Validation Loss: 0.1240
[Trial 3] Epoch 28/60, Training Loss: 0.1399, Validation Loss: 0.2356
[Trial 4] Epoch 28/60, Training Loss: 0.1273, Validation Loss: 0.1385
[Trial 5] Epoch 11/60, Training Loss: 0.1295, Validation Loss: 0.2290
[Trial 7] Epoch 2/60, Training Loss: 0.2460, Validation Loss: 0.2541
[Trial 2] Epoch 8/60, Training Loss: 0.1455, Validation Loss: 0.2340
[Trial 6] Epoch 34/60, Training Loss: 0.1289, Validation Loss: 0.1239
[Trial 7] Epoch 3/60, Training Loss: 0.1837, Validation Loss: 0.2172
[Trial 3] Epoch 29/60, Training Loss: 0.1390, Validation Loss: 0.2348
[Trial 1] Epoch 15/60, Training Loss: 0.1271, Valid

[I 2024-12-06 10:53:16,580] Trial 6 finished with value: 0.12361051738262177 and parameters: {'hidden_dim': 320, 'latent_dim': 128, 'learning_rate': 0.013478292543993551, 'batch_size': 64, 'patience': 5}. Best is trial 6 with value: 0.12361051738262177.


...

[Trial 198] Epoch 23/60, Training Loss: 0.1253, Validation Loss: 0.1260
[Trial 199] Epoch 18/60, Training Loss: 0.1251, Validation Loss: 0.1267
[Trial 199] Early stopping after 18 epochs.
[Trial 192] Epoch 37/60, Training Loss: 0.1256, Validation Loss: 0.1244
[Trial 198] Epoch 24/60, Training Loss: 0.1253, Validation Loss: 0.1259
[Trial 192] Epoch 38/60, Training Loss: 0.1256, Validation Loss: 0.1244
[Trial 198] Epoch 25/60, Training Loss: 0.1253, Validation Loss: 0.1259
[Trial 192] Epoch 39/60, Training Loss: 0.1255, Validation Loss: 0.1243
[Trial 198] Epoch 26/60, Training Loss: 0.1254, Validation Loss: 0.1259
[Trial 192] Epoch 40/60, Training Loss: 0.1258, Validation Loss: 0.1242
[Trial 198] Epoch 27/60, Training Loss: 0.1254, Validation Loss: 0.1258
[Trial 198] Epoch 28/60, Training Loss: 0.1256, Validation Loss: 0.1259
[Trial 192] Epoch 41/60, Training Loss: 0.1256, Validation Loss: 0.1243
[Trial 198] Epoch 29/60, Training Loss: 0.1252, Validation Loss: 0.1258
[Trial 192] Epoch 42

[I 2024-12-06 16:01:11,139] Trial 198 finished with value: 0.12582179270684718 and parameters: {'hidden_dim': 64, 'latent_dim': 32, 'learning_rate': 0.0018020413741597124, 'batch_size': 16, 'patience': 5}. Best is trial 67 with value: 0.12163179100801548.


[Trial 198] Epoch 34/60, Training Loss: 0.1252, Validation Loss: 0.1259
[Trial 198] Early stopping after 34 epochs.
[Trial 192] Epoch 46/60, Training Loss: 0.1260, Validation Loss: 0.1242
[Trial 192] Epoch 47/60, Training Loss: 0.1256, Validation Loss: 0.1243
[Trial 192] Epoch 48/60, Training Loss: 0.1254, Validation Loss: 0.1243
[Trial 192] Epoch 49/60, Training Loss: 0.1257, Validation Loss: 0.1242
[Trial 192] Epoch 50/60, Training Loss: 0.1258, Validation Loss: 0.1243
[Trial 192] Epoch 51/60, Training Loss: 0.1254, Validation Loss: 0.1243
[Trial 192] Epoch 52/60, Training Loss: 0.1257, Validation Loss: 0.1243
[Trial 192] Epoch 53/60, Training Loss: 0.1259, Validation Loss: 0.1243
[Trial 192] Epoch 54/60, Training Loss: 0.1258, Validation Loss: 0.1243
[Trial 192] Epoch 55/60, Training Loss: 0.1256, Validation Loss: 0.1242


[I 2024-12-06 16:01:38,020] Trial 192 finished with value: 0.12423308280607065 and parameters: {'hidden_dim': 192, 'latent_dim': 32, 'learning_rate': 0.0018458443832739656, 'batch_size': 16, 'patience': 10}. Best is trial 67 with value: 0.12163179100801548.


[Trial 192] Epoch 56/60, Training Loss: 0.1258, Validation Loss: 0.1242
[Trial 192] Early stopping after 56 epochs.

Best Trial Number: 67
Best Parameters: {'hidden_dim': 64, 'latent_dim': 32, 'learning_rate': 0.0020937734068014295, 'batch_size': 8, 'patience': 3}
Best Value (Objective): 0.12163179100801548
